# EXCEL로 만든 인공신경망을 pytorch의 Tensor 클래스을 이용하여 만들기

In [1]:
import torch

필요한 함수의 정의하기

In [11]:
def ReLU_func(outputs):
    #final_outputs=max(0,outputs)
    ReLU = torch.nn.ReLU()
    final_outputs = ReLU(outputs)
    return final_outputs

def softmax(outputs):
    # 채우기
    sm = torch.nn.Softmax(dim=-1) # 각 sample에 대해서 Softmax 하려면 dim = 1
    softmax = sm(outputs)
    return softmax

#def cross_entropy(outputs, labels):
#    cel = torch.nn.CrossEntropyLoss(reduction='none')
#   return cel(outputs, labels)

def cross_entropy(outputs, labels): 
    return -torch.sum((labels * torch.log(outputs)),dim=1)


Weight 값 설정하기

In [12]:
w_ih = torch.Tensor([[1,-2,3],[-2,5,3]])
w_ho = torch.Tensor([[3,-2,4],[-1,2,3],[2,-2,-4]])

Batch 가 3인 Input 값과 Labels 값의 입력하기

In [13]:
input = torch.Tensor([[2,-4],[0,3],[-6,2]])
labels = torch.Tensor([[1,0,0],[0,1,0],[0,0,1]])

L1의 활성화 함수 전의 값 구하기

In [14]:
L1 = torch.mm(input,w_ih) # 채우기
print(L1)

tensor([[ 10., -24.,  -6.],
        [ -6.,  15.,   9.],
        [-10.,  22., -12.]])


ReLU의 적용하기

In [15]:
L1 = ReLU_func(L1)# 채우기
print(L1)

tensor([[10.,  0.,  0.],
        [ 0., 15.,  9.],
        [ 0., 22.,  0.]])


Output 구하기

In [16]:
outputs = torch.mm(L1,w_ho)# 채우기
print(outputs)

tensor([[ 30., -20.,  40.],
        [  3.,  12.,   9.],
        [-22.,  44.,  66.]])


Softmax의 적용

In [17]:
so = softmax(outputs)
print(so)

tensor([[4.5398e-05, 8.7561e-27, 9.9995e-01],
        [1.1754e-04, 9.5246e-01, 4.7420e-02],
        [6.0546e-39, 2.7895e-10, 1.0000e+00]])


sample별로 loss 구하기

In [18]:
loss = cross_entropy(so,labels)
print(loss)

tensor([10.0000,  0.0487, -0.0000])


최종 loss 구하기

In [19]:
loss = torch.sum(loss) / input.size()[0] # 채우기
print(loss)

tensor(3.3496)


## EXCEL과 같은 답이 나오는가?

 #### 아니면 이유가 무엇인가? 


EXCEL과 다른 답이 나온다고 생각한다.

엑셀을 만드는 과정에서 정답이 (0,0,1)이고, 입력이 (-6,2)일 때 softmax layer의 3번째 노드의 값은
EXP(G7) / (EXP(G5) + EXP(G6) + EXP(G7) 이고, 이는 엑셀 상에서 1이라고 표현되는 것을 확인했다.
하지만 이를 다시 엑셀의 LN 함수를 취하면, 0이 아닌 다른 값이 나왔다. LN(1)은 0이어야 한다.

주피터 노트북에서는 0.000이라는 값이 나온 것을 확인할 수 있다. 소수점 1000자리까지 확인해도 0.0000...이다.

수많은 검색 끝에 내린 결론은, 컴퓨터에서 매우 큰 수와 매우 작은 수의 표현과 부동소수점 표현의 부정확함 때문이라고 생각한다.
실제로 소수점 자리를 30자리까지 늘려보니 1이었던 숫자가 0.9999999999721053으로 바뀌는 것을 확인할 수 있었다.

마이크로소프트 trouble shooting 정보 : (https://learn.microsoft.com/ko-kr/office/troubleshoot/excel/floating-point-arithmetic-inaccurate-result)

이러한 이유로, 엑셀에서 3번째 샘플은 e를 포함한 값으로 나오고, 파이썬은 이를 반올림하여 나타낸다.

때문에 엑셀과 주피터 노트북 파일은 서로 다른 답이 나온다고 생각한다.